In [329]:
!pip install editdistance

In [330]:
!pip install soundex

In [331]:
!pip install textdistance

In [351]:
import jaro
import heapq as hq
import editdistance
import nltk
import soundex
import textdistance
from soundex import Soundex
import pandas as pd

In [27]:
lexique = 'voc-1bwc.txt'
wrong_words = 'test-mouna.txt'

## Question 1 : Correction

In [99]:
def read_vocab(file_name):
    #lexique de mots parmi lesquels rechercher la correction (m^eme format que voc-1bwc.txt ).
    #read vocaab 
    
    with open(file_name ,'r',encoding="utf8") as f:
        lines = f.readlines()
        vocab = []
        for line in lines:       
            vocab.append(line.rstrip('\n').lstrip().split(' ')[1] )

    f.close()
    return vocab
        

In [ ]:
def get_frequency_table(file_name):

    frequency_table = {}
    # create a dict from lexique
    with open(file_name ,'r',encoding="utf8") as f:
        lines = f.readlines()

        for line in lines:
            s = line.rstrip('\n').lstrip()
            frequency = int(s.split(' ')[0])
            word = s.split(' ')[1]
            frequency_table[word] = frequency
    f.close()

    data_items = frequency_table.items()
    data_list = list(data_items)
    df = pd.DataFrame(data_list, columns=['word', 'frequency'])
    df_sorted = df.sort_values(['frequency'], ascending=False)

    return frequency_table, df_sorted


In [163]:
def find_best_candidates(vocab, scores, nb_of_candidates, largest): ### todo bug :  repetition todo fix 
    
    if largest:
     # Find three biggest scores (sorted) --> three best possible corrections
        best_scores = hq.nlargest(nb_of_candidates, scores)
    else:
        best_scores = hq.nsmallest(nb_of_candidates, scores)
            
    #find indices of highest scores
    indices_best_candidate = [i for i, x in enumerate(scores) if x == best_scores[0]] 
    indices_second_candidate = [i for i, x in enumerate(scores) if x == best_scores[1]] 
    indices_third_candidate = [i for i, x in enumerate(scores) if x == best_scores[2]] 
            
    #find the words coresponding to the scores
    corrections = [ vocab[i] for i in indices_best_candidate ] + [vocab[i] for i in  indices_second_candidate ] + [ vocab[i] for i in indices_third_candidate ]
    
    return corrections

In [332]:
def affichage(wrong_word, corrections):
    if len(corrections) == 0 : 
        result =  wrong_word + '\t' + wrong_word
    else:
        result =  wrong_word + '\t' + '\t'.join(corrections)
    print(result)

In [ ]:
def classifier(file_name, wrong_word, corrections , n=5 , ordre= 'distance' ):
    # classer ces corrections soit directement a l'aide de la ou des distances, soit a l'aide du modele unigramme que constitue le lexique passe en
    # argument au programme, soit avec les deux. Votre but est d'etudier l'impact des distances utilisees pour calculer les voisins et de decider
    # d'une meilleure approche au probleme de correction. Votre meilleure approche peut combiner plusieurs distances.

    final_corrections = []

    if ordre == 'distance':
        final_corrections = corrections[:n]

    elif ordre == 'unigram':
        frequency_table , df = get_frequency_table(file_name)

        corrections_frequency = [frequency_table[c] for c in corrections]
        corrections_frequency, corrections =  zip(*sorted(zip(corrections_frequency, corrections),reverse=True))
        final_corrections = list(corrections)[:n]

    # elif ordre == 'combination': # todo kacem
      #   final_corrections =

    affichage(wrong_word, final_corrections)

    return final_corrections




In [380]:
def corrige(wrong_words, lexique, jaro_winkler = True, Levenshtein = True,  Jaccard = True, Cosine = True, Hamming = True, LCSS = True, Damerau_levenshtein = True, soundex = True, needleman_wunsch=True):

    vocab = read_vocab(lexique)
    instance = Soundex()

    # lit une liste de mots a corriger, un par ligne

    with open(wrong_words, 'r',encoding="utf8") as f:
        for line in f:

            wrong_word = line.rstrip()
            print('********************************* ' + wrong_word)

            ###################### DISTANCE 1 : jaro_winkler distance
            if jaro_winkler :

                # calculate jaro_winkler distance for all the vocabulary
                jaro_winkler_scores = [ jaro.jaro_winkler_metric(v, wrong_word)  for v in vocab ]
                #get the best candidates
                jaro_winkler_corrections = find_best_candidates(vocab, jaro_winkler_scores, 3, True)

                #affichage
                print('Corrections using jaro_winkler distance :')
                jaro_winkler_corrections = classifier(lexique, wrong_word, jaro_winkler_corrections , ordre= 'unigram' ) # distance , ou unigram ou combination

                #affichage(wrong_word, jaro_winkler_corrections)


            ###################### DISTANCE 2 : Levenshtein distance
            if Levenshtein:

                edit_distance_scores = [ editdistance.eval(v, wrong_word)  for v in vocab ]
                edit_distance_corrections = find_best_candidates(vocab, edit_distance_scores, 3, False )

                print('Corrections using edit_distance :')
                edit_distance_corrections = classifier(lexique, wrong_word, edit_distance_corrections , ordre= 'unigram' )
                # affichage(wrong_word, edit_distance_corrections)


            ###################### DISTANCE 3 :  Jaccard Distance
            if Jaccard:
                jaccard_distance_scores = [ nltk.jaccard_distance(set(v), set(wrong_word)) for v in vocab ]
                jaccard_distance_corrections = find_best_candidates(vocab, jaccard_distance_scores, 3, False )

                print('Corrections using jaccard distance :')
                jaccard_distance_corrections = classifier(lexique, wrong_word, jaccard_distance_corrections , ordre= 'distance' )
                #affichage(wrong_word, jaccard_distance_corrections)


            ###################### DISTANCE 4 :  Cosine Distance
            if Cosine:
                cosine_distance_scores = [ textdistance.cosine.distance(v, wrong_word) for v in vocab ]
                cosine_distance_corrections = find_best_candidates(vocab, cosine_distance_scores, 3, False )

                print('Corrections using cosine_distance :')
                cosine_distance_corrections = classifier(lexique, wrong_word, cosine_distance_corrections , ordre= 'unigram' )
                #affichage(wrong_word, cosine_distance_corrections)


            ###################### DISTANCE 5 :  Hamming Distance
            if Hamming:
                hamming_distance_scores = [ textdistance.hamming.distance(v, wrong_word) for v in vocab ]
                hamming_distance_corrections = find_best_candidates(vocab, hamming_distance_scores, 3, False )

                print('Corrections using hamming distance : ')
                hamming_distance_corrections = classifier(lexique, wrong_word, hamming_distance_corrections , ordre= 'unigram' )
                #affichage(wrong_word, hamming_distance_corrections)


           ###################### DISTANCE 6 :  longest common substring similarity LCSS
            if LCSS:
                LCSS_scores = [ textdistance.lcsstr.similarity(v, wrong_word) for v in vocab ]
                LCSS_corrections = find_best_candidates(vocab, LCSS_scores, 3, True )

                print('Corrections using LCSS : ')
                LCSS_corrections = classifier(lexique, wrong_word, LCSS_corrections )
                #affichage(wrong_word, LCSS_corrections)


           ###################### DISTANCE 7 : Damerau levenshtein
            if Damerau_levenshtein:
                damerau_levenshtein_scores = [ textdistance.damerau_levenshtein.distance(v, wrong_word) for v in vocab ]
                damerau_levenshtein_corrections = find_best_candidates(vocab, damerau_levenshtein_scores, 3, False )

                print('Corrections using damerau_levenshtein : ')
                damerau_levenshtein_corrections = classifier(lexique, wrong_word, damerau_levenshtein_corrections )
                #affichage(wrong_word, damerau_levenshtein_corrections)


           ###################### DISTANCE 8 :  Soundex
            if soundex:


                # We want Englidh corrections --> remove non english corrections (-1)
                new_vocab = [ v for v in vocab if instance.compare(v, wrong_word) >= 0 ]

                Soundex_scores = [ instance.compare(v, wrong_word) for v in new_vocab ]
                Soundex_corrections = find_best_candidates(new_vocab, Soundex_scores, 3, False )

                print('Corrections using Soundex : ')
                Soundex_corrections = classifier(lexique, wrong_word, Soundex_corrections )
                #affichage(wrong_word, Soundex_corrections)


            ###################### DISTANCE 9 :  Needleman Wunsch
            if needleman_wunsch:

                needleman_wunsch_scores = [ textdistance.needleman_wunsch.distance(v, wrong_word) for v in vocab ]
                needleman_wunsch_corrections = find_best_candidates(vocab, needleman_wunsch_scores, 3, False )

                print('Corrections using needleman_wunsch : ')
                needleman_wunsch_corrections = classifier(lexique, wrong_word, needleman_wunsch_corrections )
                #affichage(wrong_word, needleman_wunsch_corrections)




    f.close()
    

In [381]:
corrige( wrong_words, lexique, jaro_winkler = True, Levenshtein = False,  Jaccard = False, Cosine = False, Hamming = False, LCSS = False, Damerau_levenshtein = False, soundex = False , needleman_wunsch=True )


********************************* aple
Corrections using jaro_winkler distance :
aple	aple	apley	apple
Corrections using Soundex : 
aple	Ã£Â©clairs	'amelio	'Ã£Â‰quipe	'arques	-a-year	'ayin	'briens	-don	'dou	ÃªÂ¼it	.edu	'escargot	'etich	.even	'hÃ£Â´te	-koh	'neill-trained	,the	'ud	.when	'acosta	Ã£Â‰dith	'afari	'afrique	'ahau	'aiguillon-sur-mer	Ã£Â‰lysÃ£Â©es	'ameh	'americain	'andratx	Ã£Â©tÃ£Â©	'berry	-do	ÃªÂ¼em	ÃªÂ¼gulf	-for	.is	'keese	'lantern	-level	'looking	-mee	-neh-zhahd	'not	.nyc	-one	'rawe	's-head	'shon	's-rights	'ss	's-wear	,they	-uh-vich	Ã£Â‰ireann	.all	'athification	'aveni	'bami	ÃªÂ¼ÃªÂ¼it	ÃªÂ¼no	'ecole	'ermilio	'errico	'heure	'horizon	'hotel	-isms	'la	'll-miss-it	'ma	'malia	-no	'orazio	.print	'rahilly	'rel	'unita	.well	-when	.why	'alsace	'ambrosi	'annunzio	'antuono	'artaix	'au	'avvenire	Ã£Â–zil	'boy	'connors	'donis	'donohue	'es	.eu	-former	.fox	'hui	-including	-kee	'mahoney	.mscief	'n'bars	'oglio	'orangerie	'saisons	-seeking	's-feet	-us	'war	.xal	'zone	'aetoa	'ahmed	'aja	'alba	

0.9111111111111111

In [317]:
textdistance.needleman_wunsch.distance('test', 'test') 

-4.0

In [245]:
# Votre programme prendra comme seul argument obligatoire le nom d'un lexique de mots 
# Vous pouvez doter votre programme d'options permettant de contr^oler le comportement de votre correcteur. 


## Evaluation

In [ ]:
reference = 'devoir3-train'

In [ ]:
def eval(correction_line, reference):
    # look for wrong word in the reference file
    
    # take the correct correctionss
    
    # compute mesure de qualite ? 
    